In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from mtcnn.mtcnn import MTCNN

In [2]:
data_folder = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba"

In [7]:
def load_images(folder_path, num_images=3):
    images = []
    labels = []
    
    count = 0  # Counter to load only the specified number of images
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder_path, filename)
            
            # Read the image
            img = cv2.imread(img_path)
            
            # Convert the image to grayscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Normalize pixel values to be between 0 and 1
            normalized_img = gray_img / 255.0
            
            # Append the image and label to the lists
            images.append(normalized_img)
            labels.append(1)  # You can set a label indicating the presence of a face
            
            count += 1
            if count == num_images:
                break
    
    return np.array(images), np.array(labels)

# Load and preprocess three images
images, labels = load_images(data_folder, num_images=10000)

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Print the shape of the training and validation sets
print("Training images shape:", train_images.shape)
print("Validation images shape:", val_images.shape)

Training images shape: (8000, 218, 178)
Validation images shape: (2000, 218, 178)


In [8]:
train_images = train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1)
val_images = val_images.reshape(val_images.shape[0], val_images.shape[1], val_images.shape[2], 1)

In [9]:
# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(train_images.shape[1], train_images.shape[2], 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification (face or not)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_images, val_labels)
print(f"Validation accuracy: {val_acc}")

Epoch 1/10
250/250 [==============================] - 85s 338ms/step - loss: 0.0028 - accuracy: 0.9994 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
250/250 [==============================] - 82s 328ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
250/250 [==============================] - 77s 309ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
250/250 [==============================] - 77s 308ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
250/250 [==============================] - 77s 307ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
250/250 [==============================] - 77s 310ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
250/250 [==============================] - 77s 308ms/step - loss: 0.0000e+0

In [10]:
model.save("face_detection_model.h5")

/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
model = load_model("face_detection_model.h5")

def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

# Load the larger image
large_image = cv2.imread("/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/pexels-pixabay-417173.jpg")

# Define the window size and step size
window_size = (218, 178)
step_size = 10

# Create a flag to indicate if a face is detected
face_detected = False

# Iterate over windows and make predictions
for (x, y, window) in sliding_window(large_image, step_size, window_size):
    # Resize the window to match the input shape expected by the model
    resized_window = cv2.resize(window, (218, 178))
    
    # Preprocess the window
    window_gray = cv2.cvtColor(resized_window, cv2.COLOR_BGR2GRAY)
    window_normalized = window_gray / 255.0
    window_input = window_normalized.reshape(1, 218, 178, 1)

    # Make predictions using the model
    prediction = model.predict(window_input)

    # Assuming 0.5 as the threshold for face detection
    if prediction[0][0] > 0.9:
        face_detected = True
        break  # Exit the loop if a face is detected

# Output True if a face is detected, False otherwise
print("Face detected:", face_detected)

1/1 [==============================] - 0s 39ms/step
Face detected: True
